In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:80% !important;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-header{margin-top:150px;}
</style>
<style>
span.toc-item-num{
display:none;
}
</style>
"""))

# 1.패키지 로드


In [2]:
#패키지 import
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

#시각화 선명도
%matplotlib inline
%config InlineBackend.figure_format= "retina"

# 한글설정
plt.rc('font',family='Malgun Gothic') # 윈도우즈
plt.rc('axes',unicode_minus=False)# 축의 -깨짐방지

# warnig 감추기
import warnings
warnings.filterwarnings(action='ignore') #안보이게
#warnings.filterwarnings(action='default') #보이게



# 2.데이터 다운로드
- https://www.data.go.kr 에서 "아파트 분양 가격 "을 검색한 csv다운로드

## ① 2013년9월부터 2015년8월까지 데이터는 df_first 변수에 담아 탐색

In [3]:
%ls D:\ai\Downloads\shareData

지정된 경로를 찾을 수 없습니다.


In [4]:
df_first = pd.read_csv(r'D:\ai\Downloads\shareData\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv',
                      encoding='cp949')
df_first

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\ai\\Downloads\\shareData\\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv'

In [ ]:
df_first.info()

- color map 참조 : https://matplotlib.org/stable/users/explain/colors/colormaps.html

In [ ]:
df_first_heat=df_first.set_index('지역')

In [ ]:
plt.figure(figsize=(25,5))
sns.heatmap(df_first_heat,annot=True,fmt='d',cmap='Greens',cbar=False)
plt.show()

## ② 이후 데이터 분양가격에 간헐적으로 comma() 추가하여 csv로 출력

In [ ]:
ex = pd.read_csv(r'D:\ai\Downloads\shareData\주택도시보증공사_전국 신규 민간아파트 분양가격 동향_20240731.csv',encoding='cp949')

In [ ]:
ex.info()

In [ ]:
# 분양가격(제곱미터) 결측치 갯수 :isna(), isnull() - 결측치 여부
ex['분양가격(제곱미터)'].isna().sum()

In [ ]:
# 분양가격(제곱미터)가 결측치인 행
ex[ex['분양가격(제곱미터)'].isna()].head()

In [ ]:
# 분양가격(제곱미터) 컬럼의 type을 float64로 타입변환: astype, pd.to_numeric()
# ex['분양가격(제곱미터)'].astype('float64')
# pd.to_numeric(ex['분양가격(제곱미터)'])
# ' ' 나 '  ' 로 인해 float64 형변환불가

In [ ]:
# 분양가격(제곱미터) 컬럼이 space인 행
ex[ex['분양가격(제곱미터)'].str.strip()==''].head(2)

In [ ]:
nanex= ex.loc[368]
spaceex=ex.loc[28]
nanex,spaceex,ex.loc[0]

In [ ]:
# 분양가격(제곱미터)이 nan이거나 space면 0, 그 외 분양가격의 문자자리수를 return 하는 함수
def get_pricelen(row):
    price= row['분양가격(제곱미터)']
    # print(row.notnull()['분양가격(제곱미터)'])
    if row.notnull()['분양가격(제곱미터)'] and price.strip()!='':
        return len(price)
    else:
        return 0 
get_pricelen(ex.loc[0]),get_pricelen(nanex),get_pricelen(spaceex)

In [ ]:
ex['가격자리수']= ex.apply(get_pricelen, axis=1)
ex.head(2)

In [ ]:
ex['가격자리수'].dtypes

In [ ]:
ex['가격자리수'].describe() # 가격자리수는 0,4,5

In [ ]:
# 가격자리수 컬럼이 0,4,5가 아닌 행
ex[~ex['가격자리수'].isin([0,4,5])]

In [ ]:
ex.head(2)

In [ ]:
# index를 컬럼으로 포함
ex.index.name = 'idx'
ex.head(2)


In [ ]:
ex = ex.reset_index()
ex.head(2)

In [ ]:
ex.loc[0]

In [ ]:
# 2021년 데이터 흉내내기 위한 함수 : nan,' ','','5123', '5,123'
def comma_append(row):
    price = row['분양가격(제곱미터)']
    if (row.idx%2==0) & (row['가격자리수']==4):
        price= price[0] +','+price[1:]
    elif (row.idx%2==0) & (row['가격자리수']==5):
        price= price[:2] +','+price[2:]
    else:
        price = price
    return price

comma_append(ex.loc[0]),comma_append(ex.loc[1]),comma_append(ex.loc[28]),comma_append(ex.loc[368])
        


In [ ]:
ex.apply(comma_append,axis=1)

In [ ]:
ex['분양가격(제곱미터)']=ex.apply(comma_append,axis=1)
ex.head()

In [ ]:
ex.drop(['idx','가격자리수'],axis=1, inplace=True)
ex.tail()

In [ ]:
ex.to_csv('data/전국제곱미터당분양가격(2015년10월부터2024년)_cp949.csv',
          encoding='cp949',
         index=False)

In [ ]:
ex.to_csv('data/전국제곱미터당분양가격(2015년10월부터2024년)_utf8.csv',
         #encoding='utf-8',
          sep='\t',
         index=False)

In [ ]:
df_last =pd.read_csv('data/전국제곱미터당분양가격(2015년10월부터2024년)_utf8.csv',
                    sep='\t')
df_last.shape

In [ ]:
df_last.head()

In [ ]:
df_last.info()

# 3.df_last의 데이터 결측치를 확인하고 대체 고민

In [ ]:
# 열별 결측시 갯수
df_last.isnull().sum()
df_last.isna().sum()

In [ ]:
# 열별 결측치를 제외한 갯수
df_last.notnull().sum()

# 4.df_last 데이터셋에 평당분양가격컬럼추가
## (1)6,222=> 6222 수정, space가 저장된경우 nan으로 수정하는 함수를 map적용
## (2)astype('float') 나 pd.to_numeric()사용

In [ ]:
# df_last['분양가격(제곱미터)'] * 3.3
# df_last.groupgy(['연도','지역명'])['분양가격(제곱미터)'].mean()에러

In [ ]:
# df_last['분양가격(제곱미터)'].astype('float64')
pd.to_numeric(df_last['분양가격(제곱미터)'],errors='coerce')

In [ ]:
type(df_last.loc[0,'분양가격(제곱미터)'])# str

In [ ]:
df_last[df_last['분양가격(제곱미터)'].isnull()].head()

In [ ]:
type(df_last.iloc[368,-1])

In [ ]:
# 분양가격이 space문자인 행수:
df_last[df_last['분양가격(제곱미터)'].str.strip()==''].shape[0]
len(df_last[df_last['분양가격(제곱미터)'].str.strip()==''])

In [ ]:
# 분양가격이 space로 되어있는 지역명과 그 데이터수
df_last.loc[df_last['분양가격(제곱미터)'].str.strip()=='','지역명'].unique()

In [ ]:
df_last['지역명'].value_counts()

In [ ]:
# 지금 현재 결측치 갯수
df_last['분양가격(제곱미터)'].isna().sum()

In [ ]:
# apply수행후 결측치 갯수
df_last['분양가격(제곱미터)'].isna().sum()+ \
len(df_last[df_last['분양가격(제곱미터)'].str.strip()==''])

### apply

In [ ]:
# 콤마가 있는 문자에서 콤마 없애기 방법1
temp='12,513'
tempsplit= temp.split(',')
print(tempsplit)
print(''.join(tempsplit))

In [ ]:
# 콤마가 있는 문자에서 콤마 없애기 방법 2
temp='12,513'
print(temp.replace(',',''))

In [ ]:
# 분양가격 컬럼에 콤마(,)가 있는 경우 콤마(,)를 제거(5,123->5123)
# 공백문자가 있는 경우 np.nan로 바꾸는 함수
def commadrop(row):
    price=row['분양가격(제곱미터)']
    if row.notnull()['분양가격(제곱미터)'] :  # 분양가격이 결측치가 아닌 경우
        if price.find(',') != -1: # 콤마제외
            #row['제곱미터당분양가격'] = price.replace(',','')
            pricesplited= price.split(',')
            row['제곱미터당분양가격'] = ''.join(pricesplited)
        elif price.strip() == '':    
            row['제곱미터당분양가격']=np.nan    
        else:
            row['제곱미터당분양가격']=price
    else : # 분양가격이 결측치인경우
        row['제곱미터당분양가격']=np.nan
    return row
commadrop(df_last.loc[0])


In [ ]:
# 공백문자는 결측치로, 콤마는 제거하기를 모든 행에 적용
df_last=df_last.apply(commadrop,axis =1)
df_last.head(3)

In [ ]:
# 결측치 갯수 : 분양가격(제곱미터)vs 제곱미터당분양가격
df_last[['분양가격(제곱미터)','제곱미터당분양가격']].isna().sum()

In [ ]:
# 공백문자 갯수 : 분양가격(제곱미터)vs 제곱미터당분양가격
df_last[df_last['분양가격(제곱미터)'].str.strip()==''].shape[0],\
len(df_last[df_last['제곱미터당분양가격'].str.strip()==''])

In [ ]:
# 콤마(,) 갯수 : 분양가격(제곱미터)vs 제곱미터당분양가격
df_last[df_last['분양가격(제곱미터)'].notnull()&\
        (df_last['분양가격(제곱미터)'].str.find(',')!=-1)].shape[0], \
df_last[df_last['제곱미터당분양가격'].notnull()&\
        (df_last['제곱미터당분양가격'].str.find(',')!=-1)].shape[0]

In [ ]:
df_last['분양가격(제곱미터)'].describe()

In [ ]:
df_last['제곱미터당분양가격'].describe()

In [ ]:
#제곱미터당분양가격 컬럼의 형변환
df_last['제곱미터당분양가격'] = df_last['제곱미터당분양가격'].astype(np.float64)
df_last['제곱미터당분양가격'] = pd.to_numeric(df_last['제곱미터당분양가격'])
df_last.head()


In [ ]:
df_last['평당분양가격'] = df_last['제곱미터당분양가격'] *3.3
df_last.head()

In [ ]:
df_last.drop(['평당분양가격','제곱미터당분양가격'],axis=1,inplace=True)
df_last.sample(1)

In [ ]:
# '3,198' ->3198 return,'3198' ->3198 return, 공백문자-> 결측치 return, 결측치-> 결측치return

def commadrop2(price):
    # np.isnan(값) : nan만 결측치로 인식하고 결측치인지 여부
    # pd.isna(값), pd.isnull(값): nan과 None을 모두 결측치로 인식하고 결측치인지 여부
    # 값 is np.nan: 값이 nan인지 여부
    if price is np.nan:
        return np.nan
    elif price.strip()== '':
        return np.nan
    elif price.find(',')!=-1:
        pricesplited = price.split(',')
        price = ''.join(pricesplited)
        return int(price)
    else:
        return int(price)
    
commadrop2(df_last.loc[0,'분양가격(제곱미터)'])# 콤마 제외한 문자를 숫자로
commadrop2(df_last.loc[1,'분양가격(제곱미터)'])
commadrop2(df_last.loc[28,'분양가격(제곱미터)'])
commadrop2(df_last.loc[368,'분양가격(제곱미터)'])

- apply(함수,axis) - 1차원 2차원(axis지정, axis 기본값 0)
- map(함수) -1차원
- applymap(함수) - 2차원 모든 요소들에 함수를 적용


In [ ]:
#df_last['분양가격(제곱미터)'].apply(commadrop2)
df_last['제곱미터당분양가격'] = df_last['분양가격(제곱미터)'].map(commadrop2)
df_last.tail(6)

In [ ]:
df_last['평당분양가격'] = df_last['제곱미터당분양가격']*3.3
df_last.head()

In [ ]:
df_last.info()

# 5. df_last 데이터셋에서 전용면적 컬럼을 추가한다 
- 전용면적 컬럼, 필요없는 컬럼 처리로 메모리 절약

In [ ]:
#방법1
df_last['전용면적'] \
    = df_last['규모구분'].apply(lambda x:x.replace('전용면적','')\
                           .replace('제곱미터','㎡')\
                           .replace('초과','~')\
                           .replace('이하','')\
                           .replace(' ',''))
df_last.head()

In [ ]:
# 방법2
df_last['전용면적'] = df_last['규모구분'].str.replace('전용면적','').str.replace('이하','').str.replace('초과','~').str.replace('제곱미터','㎡').str.replace(' ','')
df_last.head()

In [ ]:
print(df_last['규모구분'].unique())
print(df_last['전용면적'].unique())

# 6. 메모리 사용량을 줄이기 위해 불필요한 컬럼 제거

In [ ]:
df_last = df_last.drop(['규모구분','분양가격(제곱미터)','제곱미터당분양가격'],axis=1)
df_last.head(1)

In [ ]:
df_last.info()

In [ ]:
df_last[['지역명','전용면적','연도','월','평당분양가격']]
df_last.head(1)

In [ ]:
df_last.to_csv('data/df_last.csv', index = False)

# 7. GroupBy(unstack()함수이용) vs. pivot_table)

In [ ]:
df_last= pd.read_csv('data/df_last.csv')
df_last.head(1)

- df_last.groupby(["인덱스로사용할컬럼명"])["계산할컬럼명"].연산함수()
- df_last.pivot_table(index=["인덱스로사용할컬럼명"],
                   values=["계산할컬럼명"],
                   aggfunc="연산함수명")
- pd.pivot_table(df_last,index=["인덱스로사용할컬럼명"],
                   values=["계산할컬럼명"],
                   aggfunc="연산함수명")

## (1)지역별 데이터수

In [ ]:
# 지역명별 데이터 행 갯수(결측치 포함)
df_last['지역명'].value_counts() #정렬까지

In [ ]:
# 지역명별 데이터 행 갯수(결측치 제외)
df_last.groupby(['지역명'])['평당분양가격'].count()#결과가 1차원 시리즈

In [ ]:
# 지역명별 데이터 행 갯수(결측치 제외): 결과가 2차원 데이터 프레임
# 1. df_last.groupby(['지역명'])['평당분양가격'].count().to_frame().sort_values(by='평당분양가격',ascending=False)

# 2. df_last.groupby(['지역명'])['평당분양가격'].count().reset_index()
# 3. df_last.groupby(['지역명'],as_index=False)['평당분양가격'].count()
# 4. df_last.groupby([''])

In [ ]:
# 지역명별 결측치를 제외한 데이터수 (pivot_table이용 : 결과는 데이터 프레임)
df_last.pivot_table(index=['지역명'],
                   values=['평당분양가격'],
                   aggfunc='count')
pd.pivot_table(df_last, index=['지역명'],
                   values=['평당분양가격'],
                   aggfunc='count')
pd.pivot_table(df_last, index='지역명',
                   values='평당분양가격',
                   aggfunc='count').transpose()

## (2) 지역별 평당분양가격(평균,최대값,...요약통계량)

In [ ]:
df_last.groupby(['지역명'])[['평당분양가격']].mean().sort_values(by='평당분양가격',ascending=False)

In [ ]:
df_last.groupby(['지역명'])[['평당분양가격']].max().sort_values(by='평당분양가격',ascending=False).head()


In [ ]:
df_last.groupby(['지역명'])[['평당분양가격']].mean().sort_values(by='평당분양가격',ascending=False).head(3)

In [ ]:
df_last.groupby('지역명')['평당분양가격'].describe()

In [ ]:
df_last.pivot_table(index=['지역명'],
                    values=['평당분양가격'],
                   aggfunc='max').sort_values(by='평당분양가격',ascending=False).iloc[:3]

In [ ]:
df_last.pivot_table(index='지역명',
                    values='평당분양가격',
                   aggfunc='mean').sort_values(by='평당분양가격',ascending=False).iloc[:3]

In [ ]:
df_last.pivot_table(index='지역명',values = '평당분양가격', aggfunc='describe')


## (3) 전용면적별 평당분양가격(평균)

In [ ]:
df_last.groupby('전용면적')['평당분양가격'].mean()

In [ ]:
df_last.pivot_table(index ='전용면적',
                   values='평당분양가격',
                  aggfunc='mean')

## (4) 지역별, 전용면적별 평당분양가격(평균)

In [ ]:
r = df_last.groupby(['지역명','전용면적'])['평당분양가격'].mean()
r
print(r['서울'])
# 서울중 '모든면적'의 평균
print(r['서울']['모든면적'])
print(r['서울','모든면적'])

In [ ]:
r = df_last.groupby(['지역명','전용면적'])[['평당분양가격']].mean()
display(r)
display(r.loc['서울'])# 서울
# 서울 중 모든면적
display(r.loc['서울','모든면적'])# 서울
display(r.loc['서울'].loc['모든면적'])# 서울

In [ ]:
df_last.groupby(['지역명','전용면적'])[['평당분양가격']].mean().unstack()


In [ ]:
df_last.pivot_table(index=['지역명','전용면적'],
                   values='평당분양가격',
                   ).unstack()

In [ ]:
df_last['전용면적'] = df_last['전용면적'].str.replace(' ','')

In [ ]:
r = df_last.pivot_table(index='지역명',
                   columns='전용면적',
                    values='평당분양가격',
                    aggfunc='mean')
r = r[['60㎡', '60㎡~85㎡','85㎡~102㎡','102㎡~', '모든면적']]
r

In [ ]:
plt.figure(figsize=(15,5)) # 현재 plt 사이즈 조정(matplotlib,sns)

sns.heatmap(r,cmap='cool',annot=True,fmt='.0f',
           annot_kws={'color':'green','size':12,'weight':'bold'},cbar=False)

## (5) 연도, 지역별 평당 분양가격 (평균)


In [ ]:
df_last.groupby(['연도','지역명'])['평당분양가격'].mean()

In [ ]:
r= df_last.pivot_table(index=['연도','지역명'],
                   values=['평당분양가격'])
r

In [ ]:
r.unstack()

In [ ]:
r =df_last.pivot_table(index='연도',
                   columns='지역명',
                   values='평당분양가격')

In [ ]:
r.loc[2024].to_frame()

In [ ]:
r['경기'].to_frame()

In [ ]:
plt.figure(figsize=(20,5))
sns.heatmap(r,annot=True,fmt='.0f',cmap='Blues',
           annot_kws={'weight':'bold','color':'green',
                     },cbar=False)
plt.show()


# 8. 위의 도표를 시각화(matplotlib)

In [ ]:
#(1) 지역별 데이터수
result =  df_last.groupby('지역명')['평당분양가격'].count().sort_values()
result
result.min(),result.max()

In [ ]:
# result.plot(kind='line')
ax= result.plot.line(figsize=(7,2))
ax.set_title('지역별 데이터수')
ax.set_ylim([result.min()-40,result.max()+20])
plt.show()

In [ ]:
# 지역별 데이터수를  barplot
result.plot(kind='bar',rot=0,figsize=(7,2))
plt.title('지역별 데이터 수')
plt.ylim([result.min()-40,result.max()])

In [ ]:
# (2) 지역별 평당분양가격(평균,최대값,...요약통계량)
df_last.groupby('지역명')['평당분양가격'].mean().plot(kind='line',figsize=(7,2))
plt.show()

In [ ]:
# barplot

df_last.groupby('지역명')['평당분양가격'].mean().sort_values().plot(kind='bar',figsize=(7,2),rot=45)
plt.show()

In [ ]:
# (3) 전용면적별 평당분양가격(평균)
result = df_last.groupby('전용면적')['평당분양가격'].mean()

result.plot(kind='bar',rot=0,figsize=(10,3))

In [ ]:
# (3) 전용면적별 평당분양가격(barplot) -text추가(그래프안에 y값),ylim 조정
result = df_last.groupby('전용면적')['평당분양가격'].mean()
result

In [ ]:
for idx, val in enumerate(result):
    print(idx,val)

In [ ]:
ax = result.plot(kind='bar',figsize=(7,4),rot=0)
ax.set_ylim([12000,15000])
for i, val in enumerate(result):
    ax.text(i-0.18,val+200,round(val))

In [ ]:
# (4) 지역별, 전용면적별 평당분양가격(lineplot)
result = df_last.groupby('연도')['평당분양가격'].mean()
print('x축:',result.index)
print('y축:',result)

In [ ]:
for val in result:
    print(round(val),end='\t')

In [ ]:
for i in result.index:
    print(i,end='\t')

In [ ]:
for i,val in zip(result.index,result):
    print(i,val)


In [ ]:
result.plot(kind='line',figsize=(10,3))
for year in result.index:
    plt.text(year,result[year]+1000,round(result[year]))
plt.ylim([9000,21000])

In [ ]:
# 연도와 평당분양가격의 양의 상관관계를 가지는 듯 하나
# 상관관계가 0.5정도
# 연도와 평당분양가격의 변수가 scale이 맞지않아 scale 조정 필요
df_last[['연도','평당분양가격']].corr()

In [ ]:
# result.plot(line)
result.plot(kind='line',figsize=(10,3))
for year,val in zip(result.index,result):
    plt.text(year-0.3,val+100,round(val))

In [ ]:
#
result.plot(kind='bar',figsize=(10,3),rot=0)
for year,val in enumerate(result):
    plt.text(year-0.3,val+100,round(val))


In [ ]:
# 위의 연도별 평당분양가격의 boxplot
result.plot(kind='box')


In [ ]:
# 연도별, 월별 평당 분양 가격의 boxplot
result = df_last.groupby(['월','연도'])['평당분양가격'].mean().unstack()
result = df_last.pivot_table(index=['월','연도'],values='평당분양가격').unstack()
result = df_last.pivot_table(index=['월'],columns=['연도'],values='평당분양가격')
result

In [ ]:
result.plot(kind='box')

In [ ]:
plt.figure(figsize=(20,5))
sns.heatmap(result,cmap='Blues', annot=True,fmt='.0f',
           annot_kws={'weight':'bold','size':12})
plt.show()


In [ ]:
# 월별 , 연도별 평당분양가격의 추이(line plot)
result.plot(figsize=(10,4))
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
# (5) 연도, 지역별 평당 분양가격 (평균)
df_last.groupby(['연도','지역명'])['평당분양가격'].mean().unstack().plot(kind='box',figsize=(20,5))

# 9.seaborn으로 시각화
- 위에서 시각화한 내용을 seaborn으로 시각화

In [ ]:
sns.set(rc={'figure.figsize':(10,3)})
# 한글설정
plt.rc('font',family='Malgun Gothic') # 윈도우즈
plt.rc('axes',unicode_minus=False)# 축의 -깨짐방지

## (1) 지역별 평당 분양가격(평균)

In [ ]:

sns.barplot(data=df_last,x='지역명', y='평당분양가격',
            errorbar='sd'#값의 표준편차
            #ci='none'
            #ci='sd' # error
           )

plt.show()

In [ ]:
result = df_last.groupby('지역명')['평당분양가격'].mean().sort_values(ascending=False)
#result.index
sns.barplot(data=df_last,x='지역명', y='평당분양가격',order=result.index)
plt.show()

In [ ]:
# 시리즈
sns.barplot(data=result.to_frame(),x=result.index, y = '평당분양가격') # 표본평균이 아니라 표준편차가 안나옴
plt.show()

In [ ]:
for i , val in enumerate(result):
    print(i,val,end='\t')

In [ ]:
sns.barplot(x=result.index,y=result)
for i , v in enumerate(result):
    plt.text(i-0.4, v ,round(v),fontsize=9)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.barplot(data=df_last, y= '지역명',x='평당분양가격', order=result.index, errorbar=None)
for i, v in enumerate(result):
    plt.text(v+100, i+0.2 ,round(v))
plt.xlim([0,30000])


## (2) 연도별 평당 분양가격

In [ ]:
sns.barplot(data= df_last,x='연도',y='평당분양가격')
plt.show()


## (3) 그 외 lineplot,relplot,boxplot,violinplot,boxexplot,regplot,swarmplot

In [ ]:
# 년도별 평균 평당분양 가격 추이
sns.lineplot(data=df_last,x='연도',y='평당분양가격')
plt.show()

In [ ]:
# 년도별 , 지역별  평당분양가격 추이 (lineplot)
plt.figure(figsize=(12,5))
sns.lineplot(data=df_last,x='연도',y='평당분양가격',hue='지역명')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
# 위의 그래프가 겹쳐져서 잘 보이지 않아 지역명별로 분리

sns.relplot(data=df_last,x='연도',y='평당분양가격',hue='지역명',
           kind='line',col='지역명',col_wrap=4,
           facet_kws={'sharex':False,'sharey':True})
plt.show()

In [ ]:
# 연도별 평당분양가격을 지역명별로 서브플롯()
    #서브플롯: relplot(관계형 그래프의 서브플롯)
            #catplot(범주형 그래프의 서브플롯)
sns.catplot(data=df_last, x='연도' ,y='평당분양가격',
           kind='bar', col='지역명', col_wrap=4,
            # facet_kws={'sharex':False,'sharey':True},
            sharex=False)

In [ ]:
# 연도별 평당분양가격 사분위수 boxplot, boxeplot


In [ ]:
sns.violinplot(data=df_last,x='연도', y='평당분양가격')
plt.show()

In [ ]:
sns.boxenplot(data=df_last,x='연도', y='평당분양가격')
plt.show()

In [ ]:
sns.catplot(data=df_last,x='연도', y='평당분양가격',hue='지역명',
           kind='box',col='전용면적',col_wrap=4,sharex=False)
plt.show()

In [ ]:
# 연도에 따른 평당 분양가격을 회귀식으로 시각화
sns.regplot(data=df_last,x='연도', y='평당분양가격')
plt.show()

In [ ]:
sns.lmplot(data=df_last,x='연도', y='평당분양가격',hue='전용면적',col='전용면적',col_wrap=3,sharex=False)
plt.show()

In [ ]:
# swarmplot은 많은 데이터를 그리기에 시간이 많이 걸려 sampling 후 그릴것을 추천
df_sample = df_last.sample(frac=0.1,random_state=42)
df_sample

In [ ]:
sns.swarmplot(data= df_sample, x= '연도',y='평당분양가격',size=3,hue='전용면적')
plt.show()


In [ ]:
sns.swarmplot(data= df_last, x= '연도',y='평당분양가격',size=3)
plt.show()


In [ ]:
# 분양가격으로 히스토그램
df_last['평당분양가격'].hist(bins=20)

In [ ]:
sns.histplot(data=df_last,x='평당분양가격',kde=True)

In [ ]:
# 데이터 프레임의 수치형 컬럼들의 상관관계를 한 눈에 볼수 있는 pairplot
sns.pairplot(data=df_last,hue='전용면적')

In [ ]:
# 지역별 평당분양가격의 히스토그램
g = sns.FacetGrid(df_last,hue='지역명',col='지역명',col_wrap=4,sharex=False)
g.map(sns.histplot,'평당분양가격') 
plt.show()

# 10. 구조가 다른 df_last와 df_first를 연결
##  (1) concat
- new_first= df_first를 [지역명, 연도, 월, 평당분양가격]으로 변환
- new_last = df_last를 [지역명, 연도, 월, 평당분양가격]으로 변환
                (전용면적이 모든 면적인 행만 추출후 , 전용면적 열을 삭제)
        * df = pd.concat(new_first,new_last) 후, reset_index()
        * 완성된 df를 csv로 출력
              

In [ ]:
df_first

In [ ]:
df_last

In [ ]:
# (1) new_first 생성
new_first= df_first.melt(id_vars=['지역'],
                        var_name='연도월',
                        value_name='평당분양가격')
new_first

In [ ]:
#(2) 연도와 월로 분리하는 함수만들기
def parse_year(x):
    return int(x.split('년')[0])    

In [ ]:
#(3) 연도 분리
new_first['연도']=new_first['연도월'].apply(lambda x :int(x.split('년')[0]))
new_first.head()

In [ ]:
#(3) 월 분리
new_first['월']=new_first['연도월'].apply(lambda x :int(x.split('년')[-1][:-1]))
new_first.head()

In [ ]:
new_first.drop(['연도월'],axis=1,inplace=True)
new_first.head(1)

In [ ]:
new_first.columns=['지역명','평당분양가격','월','연도']

In [ ]:
new_last=df_last.loc[df_last['전용면적'].str.contains('모든면적')]

new_last.drop(['전용면적'],axis=1,inplace=True)
new_last.head()

In [ ]:
new_last

In [ ]:
new_first.head()

In [ ]:
df = pd.concat([new_first,new_last])


In [ ]:
df = df.reset_index(drop=True)
df.head(1)

In [ ]:
df.to_csv('data/df.csv', index = False)

## (2) 결측치 대체(연도별, 지역별 중위값)

In [ ]:
result = df.pivot_table(index='연도',columns='지역명',values='평당분양가격',aggfunc='median')

In [ ]:
df.loc[df['평당분양가격'].isnull()].head(2)

In [ ]:
result.loc[2018, '울산']

In [ ]:
result

In [ ]:
df.isnull().sum()

In [ ]:
def fillna_mean(row):
    if row.isnull :
        row['평당분양가격'] = result.loc[row['연도'],row['지역명']]
    return row
fillna_mean(df.loc[1010]), df.loc[1010]

In [ ]:
df = df.apply(fillna_mean,axis=1)

In [ ]:
df.to_csv('data/df.csv', index = False)

## (3)  7절부터 9절까지 분석을 한다 (전용면적별 분석은 불가)

In [ ]:
df= pd.read_csv('data/df.csv')

In [ ]:
df['지역명'].value_counts()

In [ ]:
df.groupby(['지역명'])['평당분양가격'].count()

In [ ]:
pd.pivot_table(df,index='지역명',
              values='평당분양가격',
              aggfunc='count').transpose()

In [ ]:
df.groupby(['지역명'])[['평당분양가격']].mean().sort_values(by='평당분양가격')

In [ ]:
df.groupby(['지역명'])[['평당분양가격']].max().sort_values(by='평당분양가격')

In [ ]:
df.groupby(['지역명'])[['평당분양가격']].mean().sort_values(by='평당분양가격',ascending=False).head(3)

In [ ]:
df.groupby('지역명')['평당분양가격'].describe()